# EDA
Проверить взаимосвязи между параметрами измерений, найти потенциальные особенности для построения модели.

In [ ]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from matplotlib.pyplot import rcParams

rcParams['figure.figsize'] = 16, 8